In [ ]:
%matplotlib notebook

import numpy as np
import pandas as pd
import corner
import matplotlib.pyplot as plt
%matplotlib inline

import h5py

In [ ]:
hf = h5py.File('../riz/validation.hdf5', 'r')

In [ ]:
hf.keys()

In [ ]:
columns = hf.keys()
n_cols = len(hf.keys())
n_data = 10000#len(hf['dec'][()])
arr = np.empty((n_data, n_cols))
for col_i, col in enumerate(columns):
    arr[:, col_i] = hf[col][()][:n_data]

In [ ]:
df = pd.DataFrame(arr, columns=columns)
print(df.shape)

In [ ]:
df = df.replace(np.inf, np.nan).dropna()
print(df.shape)
print(df.describe())

In [ ]:
corner_cols = ['ra', 'dec', 'mcal_mag_err_r', 'mcal_mag_err_i', 'mcal_mag_err_z', 'mcal_mag_r', 'mcal_mag_i', 'mcal_mag_z', 'redshift_true']
_ = corner.corner(df[corner_cols], 
                  labelsc=corner_cols,
                  color='tab:blue',
                 show_titles=True,
                fill_contours=True,
                levels=[0.68, 0.95, 0.997],
                range=[0.99]*len(corner_cols),
                hist_kwargs=dict(density=True, ))

In [ ]:
from sklearn import decomposition
from sklearn import datasets

X = df[['mcal_mag_err_r', 'mcal_mag_err_i', 'mcal_mag_err_z', 'mcal_mag_r', 'mcal_mag_i', 'mcal_mag_z']].values
y = df['redshift_true'].values
pca = decomposition.PCA(n_components=2)
pca.fit(X)
X_pca = pca.transform(X)

plt.scatter(X_pca[:, 0], X_pca[:, 1], marker='.', alpha=0.3, c=y)
plt.colorbar()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D


pca = decomposition.PCA(n_components=3)
pca.fit(X)
X_pca = pca.transform(X)

centers = [[1, 1], [-1, -1], [1, -1]]
plt.close('all')
fig = plt.figure(1, figsize=(8, 6))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
plt.cla()

img = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c=y, alpha=0.5, cmap='jet',
           edgecolor='k')
fig.colorbar(img)
ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])

plt.show()